In [1]:
import numpy as np
import yfinance as yf
import pandas 
from datetime import datetime
import pytz
from scipy.stats import norm

In [2]:
def get_market_data(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    stock_info = stock.info
    current_price = stock_info['regularMarketPrice']
    historical_df = stock.history(period="1y")
    expiration_dates = stock.options
    all_calls_list = []
    for expiry in expiration_dates:
        option_chain = stock.option_chain(expiry)
        calls = option_chain.calls
        calls['expirationDate'] = expiry # Add expiry date to each row
        all_calls_list.append(calls)
    
    combined_calls_df = pandas.concat(all_calls_list, ignore_index=True)
    
    
    
    combined_calls_df = pandas.concat(all_calls_list, ignore_index=True)

    market_data_dict = {
        'spot_price': current_price,
        'calls_df': combined_calls_df,
        'historical_df': historical_df
    }
    
    return market_data_dict

In [3]:
def calculate_historical_volatility(data_frame):
    data_frame['log-return'] = np.log(data_frame['Close']/data_frame['Close'].shift(1))
    daily_volatility = data_frame['log-return'].std()
    
    annual_volatility = daily_volatility * np.sqrt(252)
    
    return annual_volatility

In [4]:
def get_risk_free_rate():
    """
    Fetches the current 13-week (3-month) U.S. Treasury Bill yield as the risk-free rate.
    
    The ticker '^IRX' is for the 13-week Treasury Bill on Yahoo Finance.
    The value is returned as a decimal (e.g., 5.25% becomes 0.0525).
    
    Returns:
        float: The current risk-free rate as a decimal, or a default of 0.05 if fetching fails.
    """
    try:
        # 1. Create a Ticker object for the 13-week Treasury Bill
        irx = yf.Ticker("^IRX")
        
        # 2. Fetch the most recent historical data
        hist = irx.history(period="1d")
        
        # 3. Get the last closing value (the yield)
        # Yahoo Finance provides the rate as a percentage, so we divide by 100
        risk_free_rate = hist['Close'].iloc[-1] / 100
        
        print(f"Successfully fetched risk-free rate: {risk_free_rate:.2%}")
        return risk_free_rate
        
    except Exception as e:
        print(f"Could not fetch risk-free rate. Error: {e}")
        print("Using a default rate of 5.0%")
        return 0.05 # Return a default value if the fetch fails

In [5]:
def calculate_time_to_expiry(expiry_string):
    """
    Calculates time to expiry, correctly handling timezones and past dates.
    """
    # Convert the expiry string into a date object (ignoring time of day)
    expiry_date = datetime.strptime(expiry_string, '%Y-%m-%d').date()
    
    # Get the current date in the US market timezone (ignoring time of day)
    est = pytz.timezone('America/New_York')
    current_date = datetime.now(est).date()
    
    # Calculate the number of calendar days remaining
    days_to_expiry = (expiry_date - current_date).days
    
    # If the option expires today or has already expired,
    # return a very small positive number for T to avoid math errors.
    if days_to_expiry <= 0:
        return 1 / 365.0
    else:
        # Otherwise, return the time in years
        return days_to_expiry / 365.0
    
    

In [6]:
def black_scholes_calculator(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + np.square(sigma)/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    cost = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    
    return cost

In [7]:
#Main Function, Takes a stock ticker, uses above helper functions and values all options using helper formulas

def value_all_options(ticker_symbol):
    """
    Main function to value all call options for a ticker and compare
    theoretical price to the real market price.
    """
    # 1. Fetch all data using your primary helper
    market_data = get_market_data(ticker_symbol)
    S = market_data['spot_price']
    calls_df = market_data['calls_df']
    historical_df = market_data['historical_df']
    
    # 2. Get the single values for r and sigma using your helpers
    r = get_risk_free_rate()
    sigma = calculate_historical_volatility(historical_df)
    
    # 3. Use your helper to calculate Time to Expiry (T) for EACH option row
    calls_df['T'] = calls_df['expirationDate'].apply(calculate_time_to_expiry)

    # 4. Apply your Black-Scholes formula to every option in the DataFrame
    calls_df['theoretical_price'] = calls_df.apply(
        lambda row: black_scholes_calculator(
            S=S,
            K=row['strike'],
            T=row['T'],
            r=r,
            sigma=sigma
        ),
        axis=1 # This tells pandas to go row-by-row
    )
    
    return calls_df, S

In [8]:
if __name__ == "__main__":
    TICKER = "NVDA" # <--- Change this to any stock ticker
    
    # Get the final DataFrame with theoretical and real prices
    valued_options_df, spot_price = value_all_options(TICKER)
    
    # --- Display a clean comparison ---
    
    # Get the first available expiry date to show a focused example
    first_expiry = valued_options_df['expirationDate'].unique()[0]
    
    print(f"\n--- Price Comparison for {TICKER} (Expiry: {first_expiry}) ---")
    print(f"Current Stock Price: ${spot_price:.2f}")

    # Filter for the first expiry and sort by strike price distance to spot
    first_expiry_df = valued_options_df[valued_options_df['expirationDate'] == first_expiry]
    closest_options = first_expiry_df.iloc[(first_expiry_df['strike'] - spot_price).abs().argsort()[:15]]

    # The columns for comparison are the real price ('lastPrice') and your calculated price
    display_cols = [
        'strike', 
        'lastPrice', 
        'theoretical_price'
    ]
    print(closest_options[display_cols].sort_values(by='strike'))

Successfully fetched risk-free rate: 3.52%

--- Price Comparison for NVDA (Expiry: 2025-12-19) ---
Current Stock Price: $175.16
     strike  lastPrice  theoretical_price
225   169.0       5.75           6.350672
226   170.0       4.95           5.457272
227   171.0       3.94           4.611068
228   172.0       3.21           3.823789
229   172.5       2.89           3.455746
230   173.0       2.53           3.106431
231   174.0       1.87           2.468005
232   175.0       1.35           1.914408
233   176.0       0.94           1.447686
234   177.0       0.60           1.065809
235   177.5       0.46           0.905013
236   178.0       0.38           0.763016
237   179.0       0.22           0.530630
238   180.0       0.14           0.358160
239   181.0       0.09           0.234466
